### This notebook will only work with th data which already clean by the EDA notebook.

In [3]:
#base import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Model Import
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.optimizers import Adam
#preprocessing import
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
#accuracy calculate import
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


In [4]:
ebay_clean= pd.read_csv('../data/cleaned/Ebay_cleaned.csv', index_col=0)
pd.set_option('display.max_columns', None)
ebay_clean.head()

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,package_size,record_number,distance,handling_date,shipping_date,total_time,pay_year,pay_month,pay_date
0,1,25454,3,2019-03-27,0,0,3,5,97219,49040,13,28,1,2019-03-24,2019-03-29,5,1,1,3002,3,2,5,2019,3,24
1,0,6727381,2,2018-06-03,0,3,3,5,11415-3528,62521,0,20,1,2018-06-02,2018-06-05,0,4,2,1283,1,2,3,2018,6,2
2,1,18507,1,2019-01-08,0,4,3,5,27292,53010,1,20,1,2019-01-06,2019-01-10,9,4,3,1104,2,2,4,2019,1,6
3,1,4677,1,2018-12-18,0,0,3,5,90703,80022,1,36,1,2018-12-17,2018-12-21,8,4,4,1353,1,3,4,2018,12,17
4,1,4677,1,2018-07-28,0,0,3,5,90703,55070,1,25,1,2018-07-27,2018-07-30,3,4,5,2456,1,2,3,2018,7,27


In [5]:
ebay_clean.isna().sum()

b2c_c2c                      0
seller_id                    0
declared_handling_days       0
acceptance_scan_timestamp    0
shipment_method_id           0
shipping_fee                 0
carrier_min_estimate         0
carrier_max_estimate         0
item_zip                     0
buyer_zip                    0
category_id                  0
item_price                   0
quantity                     0
payment_datetime             0
delivery_date                0
weight                       0
package_size                 0
record_number                0
distance                     0
handling_date                0
shipping_date                0
total_time                   0
pay_year                     0
pay_month                    0
pay_date                     0
dtype: int64

In [6]:
ebay_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296420 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   b2c_c2c                    296420 non-null  int64 
 1   seller_id                  296420 non-null  int64 
 2   declared_handling_days     296420 non-null  int64 
 3   acceptance_scan_timestamp  296420 non-null  object
 4   shipment_method_id         296420 non-null  int64 
 5   shipping_fee               296420 non-null  int64 
 6   carrier_min_estimate       296420 non-null  int64 
 7   carrier_max_estimate       296420 non-null  int64 
 8   item_zip                   296420 non-null  object
 9   buyer_zip                  296420 non-null  object
 10  category_id                296420 non-null  int64 
 11  item_price                 296420 non-null  int64 
 12  quantity                   296420 non-null  int64 
 13  payment_datetime           296420 non-null  

In [38]:
X= ebay_clean[['b2c_c2c', 'declared_handling_days', 'shipment_method_id', 'shipping_fee', 'item_price', 'weight', 'package_size', 'distance']]
y= ebay_clean['total_time']

In [39]:
print(y.shape)
print(X.shape)

(296420,)
(296420, 8)


In [40]:
#split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Scale data


In [41]:
scaler= StandardScaler()
X_train_ss= scaler.fit_transform(X_train, y_train)
X_test_ss= scaler.transform(X_test)

In [42]:
y_true_test= y_test.values
y_true_train= y_train.values

**Accuracy function**

In [43]:
def define_late(y_actual, pred):
'''
    This function is using for calculate the accurancy of the model in the differnce aspect:
    when the model run, the accuracy will calculate the exacly match number for the true predict, the predict give late or early delivery will be count as false.
    But in the business point of view, The order which is delivered early than the prediction will not get any complaint from customer and will be consider as the ontime deliver.
    In this function, we will modify the accuracy of the model base on the logic above:
        day predict > actual delivered: Ontime
        day predict = actual delivered: Ontime
        day predict < actual delivered: Late

'''
    ontime= 0
    late= 0
    accuracy_sc=0
    for i in range(len(pred)):
        if pred[i] == y_actual[i] or pred[i]> y_actual[i]:
            ontime+=1
        else:
            late+=1
    accuracy_sc= ontime/len(y_actual)*100
    return accuracy_sc

**Loss Function Calulate**
$$L = \frac{1}{N}.abs([P_E.\sum_{early shipments}(actual delivery days - predicted deliveryday)+ P_L.\sum_{late shipments}(actual delivery days - predicted deliveryday)])$$
while $P_E = 0.4$, $P_L = 0.6$ and N is number of record in the dataset

In [ ]:
def evaluate_loss(preds, actual):
    ''''
        This Loss function was provided by the ebay team, who was given out the dataset for their Machine learning challenge
        From a business point of view, it is a worse experience for a buyer if a shipment arrives after the estimated delivery date (“late shipment”) 
            as compared to arriving before the estimated delivery date (“early shipment”). 
            The formula for the loss function was mentioned above.
        
    '''
    early_loss, late_loss = 0,0 
    for i in range(len(preds)):
        if preds[i] < actual[i]:
            #early shipment
            early_loss += actual[i] - preds[i]
        elif preds[i] > actual[i]:
            #late shipment
            late_loss += preds[i] - actual[i]
    loss = (1/len(preds)) * (0.4 * (early_loss) + 0.6 * (late_loss))
    return loss

#### LinearRegression model

In [44]:
#Initialize
linear_model= LinearRegression()
#fit model
linear_model.fit(X_train_ss, y_train)
linear_preds= linear_model.predict(X_test_ss)
linear_train_pred= linear_model.predict(X_train_ss)
linear_train_pred= np.round(linear_train_pred)
linear_preds= np.round(linear_preds)
linear_score_test=  linear_model.score(X_test_ss, y_test)
linear_score_train=  linear_model.score(X_train_ss, y_train)
linear_accuracy_test= define_late(y_true_test, linear_preds)
linear_accuracy_train= define_late(y_true_train, linear_train_pred)

In [45]:
print(f'Accuracy score test:  {linear_accuracy_test}')
print(f'Accuracy score train: {linear_accuracy_train}')

Accuracy score test:  70.49119492611835
Accuracy score train: 70.5704743269685


#### Ridge model

In [46]:
# Initialize
ridge_model= Ridge(solver='lsqr')

# fit
ridge_model.fit(X_train_ss, y_train)
ridge_preds= ridge_model.predict(X_test_ss)
ridge_preds= np.round(ridge_preds)
ridge_train_pred= ridge_model.predict(X_train_ss)
ridge_train_pred= np.round(ridge_train_pred)
ridge_score_test= ridge_model.score(X_test_ss, y_test)
ridge_score_train= ridge_model.score(X_train_ss, y_train)
ridge_accuracy_test= define_late(y_true_test, ridge_preds)
ridge_accuracy_train= define_late(y_true_train, ridge_train_pred)


In [47]:
print(f'Ridge Accuracy score test:  {ridge_accuracy_test}')
print(f'Ridge Accuracy score train: {ridge_accuracy_train}')

Ridge Accuracy score test:  70.48950813035557
Ridge Accuracy score train: 70.57131772484988


#### XGboost

In [48]:
# initialize 
xg_boost= XGBRegressor()
#fit
xg_boost.fit(X_train_ss, y_train)
xg_pred= xg_boost.predict(X_test_ss)
xg_pred= np.round(xg_pred)
xg_train_pred= xg_boost.predict(X_train_ss)
xg_train_pred= np.round(xg_train_pred)
xg_train_accuracy=define_late(y_true_train, xg_train_pred)
xg_test_accuracy= define_late(y_true_test, xg_pred)
print(f'Train Score: {xg_boost.score(X_train_ss, y_train)}')
print(f'test score: {xg_boost.score(X_test_ss, y_test)}')
print(f'Test Accuracy : {xg_test_accuracy}')
print(f'Train Accuracy : {xg_train_accuracy}')

Train Score: 0.27518818749163165
test score: 0.13339009103905375
Test Accuracy : 71.50833277106808
Train Accuracy : 71.6926152081506


In [50]:
print(f'rigde lost= {evaluate_loss(ridge_preds, y_true_test)} -- linear loss= {evaluate_loss(linear_preds, y_true_test)} --xg lost= {evaluate_loss(xg_pred, y_true_test)}')

rigde lost= 0.7973348626948249 -- linear loss= 0.7973449834694015 --xg lost= 0.763649551312327


In [88]:
accuracy = [linear_accuracy_test, linear_accuracy_train, ridge_accuracy_test, ridge_accuracy_train, xg_test_accuracy, xg_train_accuracy]
columns=['linear test', 'linear train', 'ridge test', 'ridge train', 'xg test', 'xg train']

#### Neural Network

In [51]:
#build model
tf.random.set_seed(123)
# Create a new sequential model
nn_model= keras.Sequential()
regularizer= keras.regularizers.l2(0.02)
#hidden layers
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
nn_model.add(Dense(40, activation="relu", kernel_regularizer=regularizer))
# nn_model.add(Dropout(0.2))
#output layer
nn_model.add(Dense(1))

#compile nn_model
nn_model.compile(
     optimizer=keras.optimizers.Adam(),
     loss=keras.losses.MeanSquaredError(),
     metrics=[keras.metrics.BinaryAccuracy()]
)

In [52]:
history= nn_model.fit(X_train_ss, y_train, epochs=50,batch_size=64, verbose=0)

In [53]:
#evaluate
NN_train_accuracy=history.history['binary_accuracy'][-1] 
result = nn_model.evaluate(X_test_ss,y_test, verbose=0)
NN_pred_test = np.round(nn_model.predict(X_test_ss))
NN_pred_train = np.round(nn_model.predict(X_train_ss))

7411/7411 [==============================] - 2s 225us/step


In [55]:
print(f'Test Accuracy: {define_late(y_true_test, NN_pred_test)}')
print(f'Train Accuracy: {define_late(y_true_train, NN_pred_train)}')

Test Accuracy: 72.20497942109169
Train Accuracy: 72.28805411240808


#### Recurrent Neural Network


In [29]:
print(X_train_ss.shape, y_train.shape)

(237136, 16) (237136,)


In [56]:
#Make embedding
number_class= X_train_ss.shape[1]
embedding_dim= 8


In [57]:
tf.random.set_seed(123)

#define rnn
rnn_mode= keras.Sequential()
# add layers
embedding_layer=Embedding(number_class, embedding_dim)

rnn_mode.add(LSTM(64, activation='relu', input_shape=(X_train_ss.shape[1], 1)))

rnn_mode.add(Dense(64, activation= 'relu'))

#output layer
rnn_mode.add(Dense(1))

# Compile mode
rnn_mode.compile(
    loss='mean_squared_error', 
    optimizer=Adam(learning_rate=0.02),
    metrics='accuracy'
)

In [58]:
# fit model
rnn_history= rnn_mode.fit(X_train_ss, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


5929/5929 [==============================] - 16s 2ms/step - loss: 9.3407 - accuracy: 0.0307 - val_loss: 9.0584 - val_accuracy: 0.0305
Epoch 2/50
5929/5929 [==============================] - 14s 2ms/step - loss: 9.3293 - accuracy: 0.0307 - val_loss: 8.4900 - val_accuracy: 0.0305
Epoch 3/50
5929/5929 [==============================] - 16s 3ms/step - loss: 12.2303 - accuracy: 0.0307 - val_loss: 8.7218 - val_accuracy: 0.0305
Epoch 4/50
5929/5929 [==============================] - 18s 3ms/step - loss: 9.4170 - accuracy: 0.0307 - val_loss: 8.7007 - val_accuracy: 0.0305
Epoch 5/50
5929/5929 [==============================] - 16s 3ms/step - loss: 9.4175 - accuracy: 0.0307 - val_loss: 8.6986 - val_accuracy: 0.0305
Epoch 6/50
5929/5929 [==============================] - 15s 3ms/step - loss: 9.4165 - accuracy: 0.0307 - val_loss: 8.6974 - val_accuracy: 0.0305
Epoch 7/50
5929/5929 [==============================] - 15s 3ms/step - loss: 9.4171 - accuracy: 0.0307 - val_loss: 8.6978 - val_accuracy: 0.

In [62]:
rnn_train_accu= rnn_history.history['accuracy'][-1]
result= rnn_mode.evaluate(X_test_ss, y_test)
rnn_pred_test= np.round(rnn_mode.predict(X_test_ss))
rnn_pred_train= np.round(rnn_mode.predict(X_train_ss))

7411/7411 [==============================] - 4s 593us/step


In [63]:
rnn_mode.summary()

Model: "sequential_1244"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense_3517 (Dense)          (None, 64)                4160      
                                                                 
 dense_3518 (Dense)          (None, 1)                 65        
                                                                 
Total params: 21,121
Trainable params: 21,121
Non-trainable params: 0
_________________________________________________________________


In [64]:
print(f'Test  accuracy= {define_late(y_true_test, rnn_pred_test)}')
print(f'Train  accuracy= {define_late(y_true_train, rnn_pred_train)}')

Test  accuracy= 71.9485864651508
Train  accuracy= 72.02533567235679


#### MPLRegressor

In [84]:
from sklearn.neural_network import MLPRegressor
network = MLPRegressor(hidden_layer_sizes = (2,3,4,3,2), activation="logistic", solver="lbfgs", max_iter=100)
network.fit(X_train_ss, y_train)
mlp_preds_test = np.round(network.predict(X_test_ss))
mlp_preds_train = np.round(network.predict(X_train_ss))

print(f'Test  accuracy= {define_late(y_true_test, mlp_preds_test)}')
print(f'Train  accuracy= {define_late(y_true_train, mlp_preds_train)}')
print(evaluate_loss(y_true_test, mlp_preds_test))

Test  accuracy= 71.9485864651508
Train  accuracy= 72.02533567235679
0.8282335874772283


So far we have tried : Linear Regression, Ridge regression, XGboost, Neural Network, and Recurrent Neural Network

The score we have for each model is:

  | Model | Accurancy Score |
  | ----------- | ----------- |
  | Linear Regression | 70.49 |
  | Ridge Regression | 70.48 |
  | XGboost | 71.51 |
  | Neural Network | 72.21 |
  | Recurrent Neural Network | 71.9486 |

At the moment, Neural Network have the best accuracy for the datase with 72.21 percent of accuracy.  We are going to do tune hyperparameter to see if we can imporve the accuracy of the model.

### Tuning Hyperparameter


##### Tuning hyperparameter for `Neural Network`

In [37]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam, RMSprop

def build_regressor(optimizer = 'adam', learning_rate=0.01):
  regressor = Sequential()
  regressor.add(Dense(units = 6 , activation= 'relu'))
  regressor.add(Dense(units = 6 , activation= 'relu'))
  regressor.add(Dense(units = 1))
  if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
  elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
     
  regressor.compile(optimizer=optimizer , loss = 'mean_squared_error' , metrics=['accuracy'])
  return regressor
KR = KerasRegressor(build_fn=build_regressor)
parameters = {'batch_size' : [32, 64, 128],
          'epochs' : [30, 40, 50],
          'optimizer':['adam','rmsprop'],
          'learning_rate': [0.001, 0.01, 0.1]}
grid_search = GridSearchCV(estimator=KR , 
param_grid=parameters,scoring='accuracy',cv=10, verbose= 0)
grid_search.fit(X_train_ss,y_train)

Epoch 1/10


/var/folders/lv/tbn_js5s77ndnb22_v4_z6q80000gn/T/ipykernel_1888/3187570055.py:20: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  KR = KerasRegressor(build_fn=build_regressor)


6670/6670 [==============================] - 2s 279us/step - loss: 8.7032 - accuracy: 0.0302
Epoch 2/10
6670/6670 [==============================] - 2s 273us/step - loss: 8.1074 - accuracy: 0.0307
Epoch 3/10
6670/6670 [==============================] - 2s 292us/step - loss: 8.0724 - accuracy: 0.0307
Epoch 4/10
6670/6670 [==============================] - 2s 296us/step - loss: 8.0194 - accuracy: 0.0307
Epoch 5/10
6670/6670 [==============================] - 2s 300us/step - loss: 8.0170 - accuracy: 0.0307
Epoch 6/10
6670/6670 [==============================] - 2s 298us/step - loss: 8.0086 - accuracy: 0.0307
Epoch 7/10
6670/6670 [==============================] - 2s 279us/step - loss: 8.0077 - accuracy: 0.0307
Epoch 8/10
6670/6670 [==============================] - 2s 301us/step - loss: 8.0011 - accuracy: 0.0307
Epoch 9/10
6670/6670 [==============================] - 2s 282us/step - loss: 8.0000 - accuracy: 0.0307
Epoch 10/10
742/742 [==============================] - 0s 195us/step
Epoch 

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

6670/6670 [==============================] - 2s 281us/step - loss: 8.7521 - accuracy: 0.0305
Epoch 2/10
6670/6670 [==============================] - 2s 277us/step - loss: 8.1559 - accuracy: 0.0308
Epoch 3/10
6670/6670 [==============================] - 2s 281us/step - loss: 8.1162 - accuracy: 0.0308
Epoch 4/10
6670/6670 [==============================] - 2s 274us/step - loss: 8.0921 - accuracy: 0.0308
Epoch 5/10
6670/6670 [==============================] - 2s 276us/step - loss: 8.0820 - accuracy: 0.0308
Epoch 6/10
6670/6670 [==============================] - 2s 286us/step - loss: 8.0770 - accuracy: 0.0308
Epoch 7/10
6670/6670 [==============================] - 2s 292us/step - loss: 8.0695 - accuracy: 0.0308
Epoch 8/10
6670/6670 [==============================] - 2s 290us/step - loss: 8.0613 - accuracy: 0.0308
Epoch 9/10
6670/6670 [==============================] - 2s 275us/step - loss: 8.0642 - accuracy: 0.0308
Epoch 10/10
742/742 [==============================] - 0s 190us/step
Epoch 

...

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

6670/6670 [==============================] - 3s 404us/step - loss: 9.0367 - accuracy: 0.0308
Epoch 2/40
6670/6670 [==============================] - 3s 400us/step - loss: 9.0362 - accuracy: 0.0308
Epoch 3/40
6670/6670 [==============================] - 3s 401us/step - loss: 9.0047 - accuracy: 0.0308
Epoch 4/40
6670/6670 [==============================] - 3s 415us/step - loss: 8.9858 - accuracy: 0.0308
Epoch 5/40
6670/6670 [==============================] - 3s 395us/step - loss: 9.4203 - accuracy: 0.0308
Epoch 6/40
6670/6670 [==============================] - 3s 398us/step - loss: 9.3895 - accuracy: 0.0308
Epoch 7/40
6670/6670 [==============================] - 3s 395us/step - loss: 9.3887 - accuracy: 0.0308
Epoch 8/40
6670/6670 [==============================] - 3s 409us/step - loss: 9.3918 - accuracy: 0.0308
Epoch 9/40
6670/6670 [==============================] - 3s 398us/step - loss: 9.3874 - accuracy: 0.0308
Epoch 10/40
6670/6670 [==============================] - 3s 398us/step - lo

/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 192, in wrapper
    return func(*args, **kwargs)
  File "/Users/henryvuong/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classif

6670/6670 [==============================] - 3s 395us/step - loss: 9.0021 - accuracy: 0.0305
Epoch 2/40
6670/6670 [==============================] - 3s 403us/step - loss: 9.7725 - accuracy: 0.0305
Epoch 3/40
6670/6670 [==============================] - 3s 375us/step - loss: 9.3348 - accuracy: 0.0305
Epoch 4/40
6670/6670 [==============================] - 3s 380us/step - loss: 9.3349 - accuracy: 0.0305
Epoch 5/40
6670/6670 [==============================] - 3s 375us/step - loss: 9.3312 - accuracy: 0.0305
Epoch 6/40
6670/6670 [==============================] - 3s 386us/step - loss: 9.3346 - accuracy: 0.0305
Epoch 7/40
6670/6670 [==============================] - 3s 377us/step - loss: 9.3341 - accuracy: 0.0305
Epoch 8/40
6670/6670 [==============================] - 3s 382us/step - loss: 9.3355 - accuracy: 0.0305
Epoch 9/40
6670/6670 [==============================] - 2s 373us/step - loss: 9.3335 - accuracy: 0.0305
Epoch 10/40
6670/6670 [==============================] - 2s 372us/step - lo

KeyboardInterrupt: 

In [184]:
grid_search.best_params_

{'batch_size': 64, 'epochs': 50, 'optimizer': 'adam'}

##### Turning Hyperparameter for `Ridge` and `XGboost`

In [88]:
import tensorflow as tf
tf.keras.backend.clear_session()
estimators= [
    ('normalise', StandardScaler()),
    ('model', LinearRegression())
] 
np.random.seed(123)

my_pipe= Pipeline(estimators)

grid1= [
    {
        'model': [Ridge()],
        'normalise':[StandardScaler()],
        'model__alpha':[0.001, 0.01, 0.1, 1, 10],
        'model__solver':['auto']
    }]
grid2=[    
    {
        'model':[XGBRegressor()],
        'normalise':[StandardScaler()],
        'model__subsample': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'model__colsample_bytree':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'model__max_depth': range (5, 15, 1),
        'model__min_child_weight':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
        'model__n_estimators': range(60, 220, 40),
        'model__learning_rate': [0.1, 0.01, 0.05]
    }
]
gridCV1= GridSearchCV(my_pipe, grid1, cv=5, verbose=0)
gridCV2= GridSearchCV(my_pipe, grid2, cv=5, verbose=0)

fit_grid1= gridCV1.fit(X_train_ss, y_train)
fit_grid2= gridCV2.fit(X_train_ss, y_train)

KeyboardInterrupt: 

In [36]:
fit_grid.best_estimator_

Pipeline(steps=[('normalise', StandardScaler()),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=180,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [37]:
fit_grid.best_params_

{'model': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=180, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...),
 'model__learning_rate': 0.05,
 'model__max_depth': 6,
 'model__n_estimators': 180,
 'normalise': StandardScaler()}

In [38]:
fit_grid.best_score_

0.1565110196423511

In [39]:
fit_grid.predict(X_test)

array([3.4897463, 4.0850334, 3.625931 , ..., 3.8562133, 4.971652 ,
       3.5020971], dtype=float32)

In [58]:
print(f'Test Accuracy: {define_late(np.round(fit_grid.predict(X_test)), y_true_test)}')
print(f'Train Accuracy: {define_late(np.round(fit_grid.predict(X_train)), y_true_train)}')

Test Accuracy: 71.52520072869576
Train Accuracy: 71.66857836853114
